In [1]:
pip install spotipy

   ---------------------------------------- 0.0/261.2 kB ? eta -:--:--
   --------------------- ------------------ 143.4/261.2 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 261.2/261.2 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [34]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id = "5eb991d290a34252b8bb360422ed4e5f",client_secret="c9f1a87c66594bbda071c622f174db75")

In [5]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [6]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF"

In [13]:
playlist_URI = playlist_link.split("/")[-1]

In [25]:
data = sp.playlist_tracks(playlist_URI)

In [28]:
# Writing the logic(for loop) to extract only necessary data and save it in a variable
album_list = []
for row in data['items']:
    album_id = row['track']['album']['id']
    album_name = row['track']['album']['name']
    album_release_date = row['track']['album']['release_date']
    album_total_tracks = row['track']['album']['total_tracks']
    album_url = row['track']['album']['external_urls']['spotify']
    album_element = {'album_id':album_id,'name':album_name,'release_date':album_release_date,'total_tracks':album_total_tracks,'url':album_url}

    album_list.append(album_element)




In [33]:
# Extracting song_list Data

song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    album_id = row['track']['album']['id']
    artist_id = row['track']['album']['artists'][0]['id']
    song_element = {'song_id':song_id,'song_name':song_name,'duration_ms':song_duration,'url':song_url,'popularity':song_popularity,'song_added':song_added,
                   'album_id':album_id,'artist_id':artist_id
                   }
    song_list.append(song_element)

In [41]:
artist_list = []
for row in data['items']:
    for key,value in row.items():
        if key == "track":
            for artist in value['artists']:
                artist_dict = {'artist_id':artist['id'],'artist_name':artist['name'],'external_url':artist['href']}
                artist_list.append(artist_dict)

In [42]:
album_df = pd.DataFrame.from_dict(album_list)
artist_df = pd.DataFrame.from_dict(artist_list)
song_df = pd.DataFrame.from_dict(song_list)

In [51]:
# Changing Object DataType To Datatime DataType For Less Redundancy and Appropriate DataType
album_df['release_date'] = pd.to_datetime(album_df['release_date'], errors='coerce')
song_df['song_added'] = pd.to_datetime(song_df['song_added'], errors='coerce')
artist_df['song_added'] = pd.to_datetime(song_df['song_added'], errors='coerce')

In [37]:
album_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   album_id      50 non-null     object
 1   name          50 non-null     object
 2   release_date  50 non-null     object
 3   total_tracks  50 non-null     int64 
 4   url           50 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


In [52]:
# Dropping The Duplicates
album_df = album_df.drop_duplicates(subset= ['album_id'])
artist_df = artist_df.drop_duplicates(subset= ['artist_id'])
song_df = song_df.drop_duplicates(subset= ['song_id'])
